https://www.kaggle.com/competitions/contradictory-my-dear-watson

# Importing Libs and paths

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB #3 veya daha fazla seçenek için
from sklearn.linear_model import LogisticRegression# 2 seçenek varsa kullanılır
from textblob import TextBlob , Word
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

# Installing requırments

In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

2022-08-03 12:21:53.497519: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-08-03 12:21:53.497659: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

2022-08-03 12:21:58.650118: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-03 12:21:58.652692: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-08-03 12:21:58.652737: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-03 12:21:58.652768: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (4c304588605b): /proc/driver/nvidia/version does not exist
2022-08-03 12:21:58.655462: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

# Reading data

In [6]:
sub_df=pd.read_csv("../input/contradictory-my-dear-watson/sample_submission.csv")
df2=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")
df1=pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")

# EDA

In [7]:
df1.tail()

,id,premise,hypothesis,lang_abv,language,label
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2
12119,44301dfb14,"For himself he chose Atat??rk, or Father of th...",Ataturk was the father of the Turkish nation.,en,English,0


In [8]:
df1.shape

(12120, 6)

In [9]:
df2.tail()

,id,premise,hypothesis,lang_abv,language
5190,5f90dd59b0,نیند نے وعدہ کیا کہ موٹل نے سوال میں تحقیق کی.,نیمیتھ کو موٹل کی تفتیش کے لئے معاوضہ دیا جارہ...,ur,Urdu
5191,f357a04e86,The rock has a soft texture and can be bough...,The rock is harder than most types of rock.,en,English
5192,1f0ea92118,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,她在与Wolverstone的打斗结束后才在场的事实被看作是很尴尬的。,zh,Chinese
5193,0407b48afb,isn't it i can remember i've only been here ei...,I could see downtown Dallas from where I lived...,en,English
5194,16c2f2ab89,"In Hong Kong you can have a plate, or even a w...",It's impossible to have a plate hand-painted t...,en,English


In [10]:
df2.shape

(5195, 5)

In [11]:
df=df1.append(df2)

In [12]:
df.tail()

,id,premise,hypothesis,lang_abv,language,label
5190,5f90dd59b0,نیند نے وعدہ کیا کہ موٹل نے سوال میں تحقیق کی.,نیمیتھ کو موٹل کی تفتیش کے لئے معاوضہ دیا جارہ...,ur,Urdu,NaN
5191,f357a04e86,The rock has a soft texture and can be bough...,The rock is harder than most types of rock.,en,English,NaN
5192,1f0ea92118,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,她在与Wolverstone的打斗结束后才在场的事实被看作是很尴尬的。,zh,Chinese,NaN
5193,0407b48afb,isn't it i can remember i've only been here ei...,I could see downtown Dallas from where I lived...,en,English,NaN
5194,16c2f2ab89,"In Hong Kong you can have a plate, or even a w...",It's impossible to have a plate hand-painted t...,en,English,NaN


# Deep understanding and feature engineering

In [13]:
df["lang_abv"].value_counts()

en    9815
zh     562
ar     560
sw     557
ur     549
ru     548
fr     547
es     541
el     540
th     535
hi     524
vi     524
tr     518
de     503
bg     492
Name: lang_abv, dtype: int64

In [14]:
df["lang_abv"].unique(),df["language"].unique()

(array(['en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh', 'hi',
        'sw', 'vi', 'es', 'el'], dtype=object),
 array(['English', 'French', 'Thai', 'Turkish', 'Urdu', 'Russian',
        'Bulgarian', 'German', 'Arabic', 'Chinese', 'Hindi', 'Swahili',
        'Vietnamese', 'Spanish', 'Greek'], dtype=object))

In [15]:
for i in list(df["lang_abv"].unique()):
    df_hyp=df.loc[df["lang_abv"]==i]
    df_hyp["hypothesis"]=df_hyp["hypothesis"].str.lower()
    df_hyp["hypothesis"]=df_hyp["hypothesis"].str.replace("[^\w\s]","")
    df_hyp["hypothesis"]=df_hyp["hypothesis"].str.replace("\d+","")
    df_hyp["hypothesis"]=df_hyp["hypothesis"].str.replace("\n"," ").str.replace("\r","")
    df.loc[df["lang_abv"]==i]=df_hyp

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing

In [16]:
for i in list(df["lang_abv"].unique()):
    df_pre=df[df["lang_abv"]==i]
    df_pre["premise"]=df_pre["premise"].str.lower()
    df_pre["premise"]=df_pre["premise"].str.replace("[^\w\s]","")
    df_pre["premise"]=df_pre["premise"].str.replace("\d+","")
    df_pre["premise"]=df_pre["premise"].str.replace("\n"," ").str.replace("\r","")
    df[df["lang_abv"]==i]=df_pre

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing

In [17]:
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,the rules developed in the interim were put to...,en,English,0.0
1,5b72532a0b,these are issues that we wrestle with in pract...,practice groups are not permitted to work on t...,en,English,2.0
2,3931fbe82a,des petites choses comme celleslà font une dif...,jessayais daccomplir quelque chose,fr,French,0.0
3,5622f0c60b,you know they cant really defend themselves li...,they cant defend themselves because of their age,en,English,0.0
4,86aaa48b45,ในการเลนบทบาทสมมตกเชนกน โอกาสทจะไดแสดงออกและได...,เดกสามารถเหนไดวาชาตพนธแตกตางกนอยางไร,th,Thai,1.0


In [18]:
pip install stop-words

Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
import numpy as np
# Plotting packs
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#NLP packs
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from stop_words import get_stop_words
import re 
import string

##Wordcloud Modules
from wordcloud import WordCloud , STOPWORDS
from wordcloud import WordCloud
#Machine learning Packs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [23]:
lang_list=df.language.unique().tolist()

In [24]:
import stop_words
list_stop=stop_words.AVAILABLE_LANGUAGES

In [25]:
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,the rules developed in the interim were put to...,en,English,0.0
1,5b72532a0b,these are issues that we wrestle with in pract...,practice groups are not permitted to work on t...,en,English,2.0
2,3931fbe82a,des petites choses comme celleslà font une dif...,jessayais daccomplir quelque chose,fr,French,0.0
3,5622f0c60b,you know they cant really defend themselves li...,they cant defend themselves because of their age,en,English,0.0
4,86aaa48b45,ในการเลนบทบาทสมมตกเชนกน โอกาสทจะไดแสดงออกและได...,เดกสามารถเหนไดวาชาตพนธแตกตางกนอยางไร,th,Thai,1.0


In [26]:
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

for a in df["lang_abv"].unique().tolist():
  datas=df[df["lang_abv"]==a]
  langu=datas["language"].values.tolist()[0].lower()
  texts=datas["premise"].values.tolist()
  data=texts
  clean_txt=[]
  for i in range(0,len(data)):
      text=data[i]
      text=text.split()
      ls=LancasterStemmer()#kelimelerden ekleri çıkararak kök bulma
      if langu in list_stop:
        text=[ls.stem(j)for j in text if not j in set(stop_words.get_stop_words(langu))]
      else:
        text=[ls.stem(j)for j in text]
      lem=WordNetLemmatizer()#kelimelerden ekleri çıkararak kök bulma
      if langu in list_stop:
        text=[lem.lemmatize(j)for j in text if not j in set(stop_words.get_stop_words(langu))]
      else:
        text=[lem.lemmatize(j)for j in text ]
      
      #stopwords kaldırma   
      text=" ".join(text)#kelimeleri yeniden birleştirme
      clean_txt.append(text)
  df.loc[df["lang_abv"]==a,"premise"]=clean_txt

In [27]:
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,com consid form interim rul,the rules developed in the interim were put to...,en,English,0.0
1,5b72532a0b,issu wrestl pract group law firm said,practice groups are not permitted to work on t...,en,English,2.0
2,3931fbe82a,petit chos celleslà différ énorm jessay,jessayais daccomplir quelque chose,fr,French,0.0
3,5622f0c60b,know cant real defend lik somebody grown uh sa...,they cant defend themselves because of their age,en,English,0.0
4,86aaa48b45,ในการเลนบทบาทสมมตกเชนกน โอกาสทจะไดแสดงออกและได...,เดกสามารถเหนไดวาชาตพนธแตกตางกนอยางไร,th,Thai,1.0


In [28]:
for a in df["lang_abv"].unique().tolist():
  datas=df[df["lang_abv"]==a]
  langu=datas["language"].values.tolist()[0].lower()
  texts=datas["hypothesis"].values.tolist()
  data=texts
  clean_txt=[]
  for i in range(0,len(data)):
      text=data[i]
      text=text.split()
      ls=LancasterStemmer()#kelimelerden ekleri çıkararak kök bulma
      if langu in list_stop:
        text=[ls.stem(j)for j in text if not j in set(stop_words.get_stop_words(langu))]
      else:
        text=[ls.stem(j)for j in text]
      lem=WordNetLemmatizer()#kelimelerden ekleri çıkararak kök bulma
      if langu in list_stop:
        text=[lem.lemmatize(j)for j in text if not j in set(stop_words.get_stop_words(langu))]
      else:
        text=[lem.lemmatize(j)for j in text ]
      
      #stopwords kaldırma   
      text=" ".join(text)#kelimeleri yeniden birleştirme
      clean_txt.append(text)
  df.loc[df["lang_abv"]==a,"hypothesis"]=clean_txt

In [29]:
df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,com consid form interim rul,rul develop interim put togeth com mind,en,English,0.0
1,5b72532a0b,issu wrestl pract group law firm said,pract group permit work issu,en,English,2.0
2,3931fbe82a,petit chos celleslà différ énorm jessay,jessaya daccomplir quelqu chos,fr,French,0.0
3,5622f0c60b,know cant real defend lik somebody grown uh sa...,cant defend ag,en,English,0.0
4,86aaa48b45,ในการเลนบทบาทสมมตกเชนกน โอกาสทจะไดแสดงออกและได...,เดกสามารถเหนไดวาชาตพนธแตกตางกนอยางไร,th,Thai,1.0


In [30]:
test=df[df["label"].isnull()]
train=df[df["label"].notnull()]

In [31]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [32]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

In [33]:
max_len = 200

In [34]:
def bert_encode(hypotheses, premises, tokenizer, max_len=100):

  #Token embeddings
  sentence1 = tf.ragged.constant([
      encode_sentence(s)
       for s in np.array(hypotheses)])
  sentence2 = tf.ragged.constant([
      encode_sentence(s)
       for s in np.array(premises)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
  input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)
  pad = max_len-input_word_ids.to_tensor().shape[1]
  if pad > 0:
      pad_seq = tf.zeros((sentence1.shape[0], pad),dtype=np.int32)
      input_word_ids = tf.concat([input_word_ids, pad_seq], axis=-1)

  # The input mask to allow the model distinguish between the content and the padding.
  input_mask = tf.ones_like(input_word_ids)
  
  # Sentence embedding
  type_cls = tf.zeros_like(cls)
  type_s1 = tf.zeros_like(sentence1)
  type_s2 = tf.ones_like(sentence2)
  input_type_ids = tf.concat(
      [type_cls, type_s1, type_s2], axis=-1)
  if pad > 0:
      pad_seq = tf.zeros((sentence1.shape[0], pad),dtype=np.int32)
      input_type_ids = tf.concat([input_type_ids, pad_seq], axis=-1)
  
  input_type_ids = input_type_ids.to_tensor()
  input_mask = input_mask.to_tensor()
  input_word_ids = input_word_ids.to_tensor()
  inputs = {
      'input_word_ids': input_word_ids[:,:max_len],
      'input_mask': input_mask[:,:max_len],
      'input_type_ids': input_type_ids[:,:max_len]
      }
  print('input_word_ids', inputs['input_word_ids'].shape)
  print('input_mask', inputs['input_mask'].shape)
  print('input_type_ids', inputs['input_type_ids'].shape)
  return inputs

In [35]:
train_input = bert_encode(train.premise.values, train.hypothesis.values, tokenizer,max_len)


input_word_ids (12120, 200)
input_mask (12120, 200)
input_type_ids (12120, 200)


In [36]:
test_input = bert_encode(test.premise.values, test.hypothesis.values, tokenizer,max_len)

input_word_ids (5195, 200)
input_mask (5195, 200)
input_type_ids (5195, 200)


In [37]:

def build_model():
    bert_encoder = TFBertModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
    
    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [38]:
with strategy.scope():
    model = build_model()
    model.summary()

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 200)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 177853440   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [39]:
model.fit(train_input,
          train.label.values, 
          epochs = 20, 
          verbose = 1, 
          batch_size = 64, 
          validation_split = 0.2)

Epoch 1/20
152/152 [==============================] - 145s 479ms/step - loss: 1.0873 - accuracy: 0.3866 - val_loss: 1.0052 - val_accuracy: 0.5074
Epoch 2/20
152/152 [==============================] - 16s 103ms/step - loss: 0.9755 - accuracy: 0.5224 - val_loss: 0.9660 - val_accuracy: 0.5243
Epoch 3/20
152/152 [==============================] - 16s 104ms/step - loss: 0.8769 - accuracy: 0.5963 - val_loss: 0.9807 - val_accuracy: 0.5248
Epoch 4/20
152/152 [==============================] - 16s 104ms/step - loss: 0.7810 - accuracy: 0.6560 - val_loss: 1.0528 - val_accuracy: 0.5182
Epoch 5/20
152/152 [==============================] - 16s 104ms/step - loss: 0.6405 - accuracy: 0.7255 - val_loss: 1.2496 - val_accuracy: 0.5037
Epoch 6/20
152/152 [==============================] - 16s 104ms/step - loss: 0.5323 - accuracy: 0.7744 - val_loss: 1.3319 - val_accuracy: 0.4901
Epoch 7/20
152/152 [==============================] - 16s 104ms/step - loss: 0.3936 - accuracy: 0.8411 - val_loss: 1.4664 - val_a

In [40]:
predictions = [np.argmax(i) for i in model.predict(test_input)]

In [41]:
submission = test.id.copy().to_frame()
submission['prediction'] = predictions

In [42]:
submission.head()

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,1


In [43]:
submission.to_csv("submission.csv", index = False)